In [1]:
# Import, authenticate and initialize the Earth Engine library
import ee
# ee.Authenticate()
ee.Initialize()

In [3]:
# Import modules
import pandas as pd
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import *
!pip install geopandas --quiet
import geopandas as gpd
import numpy as np
# !pip install datatable --quiet
!pip install git+https://github.com/h2oai/datatable --quiet
# !pip install datatable --quiet
import datatable as dt
from datatable import f
import time
from pprint import pprint
import os, re
!pip install gee_subset --quiet
from gee_subset import gee_subset
import shapely.geometry
import re
!pip install geemap --quiet
import geemap
!pip install git+https://github.com/loicdtx/landsat-extract-gee --quiet
# !pip install geextract
import geextract
from geextract import ts_extract

You should consider upgrading via the '/home/shoyo/carbon/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/shoyo/carbon/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/shoyo/carbon/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/shoyo/carbon/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/shoyo/carbon/bin/python3.9 -m pip install --upgrade pip' command.


In [5]:
os.getcwd()
os.chdir('/home/shoyo/Desktop')

csv_path = '/SOC_lonlatlandcover.csv'
df = pd.read_csv(os.getcwd() + csv_path)
df = df.drop(columns=['other'])
df

,Unnamed: 0,latitude,longitude,year,predsoc,bushland,cropland,forest,grassland,mangrove,shrubland,thicket,wooded_grassland,woodland
0,0,1.378330,35.191816,2021,6.84225,0,1,0,0,0,0,0,0,0
1,1,1.385067,35.193972,2021,5.94385,0,1,0,0,0,0,0,0,0
2,2,1.384528,35.195589,2021,4.45060,0,0,0,0,0,0,0,1,0
3,3,1.378869,35.190468,2021,7.10625,0,0,0,0,0,0,0,1,0
4,4,1.379678,35.187234,2021,7.06505,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2631,2631,0.154286,34.343447,2021,11.69960,0,1,0,0,0,0,0,0,0
2632,2632,0.158328,34.342908,2021,17.27550,0,1,0,0,0,0,0,0,0
2633,2633,0.159676,34.342099,2021,16.55060,1,0,0,0,0,0,0,0,0
2634,2634,0.160754,34.343177,2021,11.26630,0,1,0,0,0,0,0,0,0


In [37]:
def cloudMask(image):
  
  qa = image.select('QA_PIXEL')
  dilated = 1 << 1
  cirrus = 1 << 2
  cloud = 1 << 3
  shadow = 1 << 4

  # mask = ((qa.bitwise_and(dilated) == 0) & (qa.bitwise_and(cirrus) == 0) & (qa.bitwise_and(cloud) == 0) & (qa.bitwise_and(shadow) == 0))
  # mask = ((qa.bitwise_and(1 << 1) == 0) & (qa.bitwise_and(1 << 2) == 0) & (qa.bitwise_and(1 << 3) == 0) & (qa.bitwise_and(1 << 4) == 0))
  mask = ((qa.bitwise_and(1 << 3).neq(0)).multiply(1)
      .add((qa.bitwise_and(1 << 4).neq(0)).multiply(1))
      # .add((qa.bitwise_and(1 << 2).neq(0)).multiply(1))
      # .add((qa.bitwise_and(1 << 1).neq(0)).multiply(1))
      )

  return image.updateMask(mask)

def addEVI(image):

  SR = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  image = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  image = image.addBands(SR, None, True)

  evi = image.expression( '2.5 * ( (NIR-RED) / (NIR + 6*RED - 7.5*BLUE + 1))', {
      'NIR': image.select('SR_B5'),
      'RED': image.select('SR_B4'),
      'BLUE': image.select('SR_B2'),
      }).rename('EVI')
  return image.addBands(evi)

def get_landsat(lat, lon, start, end):
  point = ee.Geometry.Point(lon, lat).buffer(15).bounds()
  landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(start, end).filterBounds(point).map(cloudMask).map(addEVI).select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'EVI', 'ST_B10'])
  landsat9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterDate(start, end).filterBounds(point).map(cloudMask).map(addEVI).select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'EVI', 'ST_B10'])
  # landsat = landsat8.merge(landsat9).reduce(ee.Reducer.median(), bestEffort=True)
  landsat = landsat8.merge(landsat9).median()
  # landsat = landsat.updateMask(landsat.mask())
  landsat = ee.ImageCollection.fromImages([landsat]).getRegion(point, 30).getInfo()

  return landsat

In [38]:
dataframe = pd.DataFrame(columns=['latitude', 'longitude', 'year', 'SOC', 'bushland', 'cropland',
       'forest', 'grassland', 'mangrove', 'shrubland', 'thicket',
       'wooded_grassland', 'woodland', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'EVI', 'ST_B10'])

for i in range(len(df)):
  print(i)
  for j in range(6):
    elem = get_landsat(df.iloc[i, 0], df.iloc[i, 1], 
                       ee.Date(str(df.iloc[i, 2]) + '-01-01').advance(-2, 'year').advance(6*j, 'month'), 
                       ee.Date(str(df.iloc[i, 2]) + '-06-30').advance(-2, 'year').advance(6*j, 'month')
                       )
    # print(elem)

    newRow = pd.Series( df.iloc[i, :13].to_list() + [ elem[1][l] for l in range(4, 12) ],
                       index=dataframe.columns )
    
    if newRow.isna().any() == True:
      print(newRow)
    dataframe = dataframe.append(newRow, ignore_index=True)
dataframe

0


NameError: name 'ee' is not defined

In [ ]:
reference = dataframe[dataframe.isna().any(axis=1)].loc[:, ['latitude','longitude']]
reference

In [ ]:
dftrue = pd.DataFrame(columns=dataframe.columns)
for i in range(len(dataframe)):
    lat = dataframe.loc[i, 'latitude']
    lon = dataframe.loc[i, 'longitude']
    if not ((reference['latitude'] == lat) & (reference['longitude'] == lon)).any():
        dftrue = dftrue.append(dataframe.loc[i])

In [ ]:
dftrue[dftrue.isna().any(axis=1)]

In [ ]:
dftrue.to_csv('/SOClandsat6mTrue.csv', index=False)

In [ ]:
reference.to_csv('/SOClandsat6mFalse.csv', index=False)